# Disclaimer
Most of the code in this repository is based on the code provided by the original article and is not my original work. I have only added some additional experiments and explanations. The original code can be found [here](https://github.com/usaito/unbiased-implicit-rec-real).

# Introduction
In this notebook, I will be partially reproducing the experiments provided by the original researchers. The steps are as follows:
* Train a model on the MovieLens 100k dataset
* Plot and analyse the results

# Imports and setup

In [7]:
import codecs
import yaml
from pathlib import Path
from typing import Tuple

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.framework import ops


# Train a model
In this part, we will train a model using semi-synthetic data. It uses the MovieLens 100K dataset, that can be fond [here](https://grouplens.org/datasets/movielens/100k/). (Note that in this repository, I have only left the relevant files, and not the whole dataset.)

This data contains five-star movie ratings and the ratings are MNAR. As the article states, to facilitate ground-truth evaluation against a fully known relevance and exposure parameters, we can create parameters as follows:
1. Using Rating-based factorization we found an approximation of the true ratings as $R_{u,i} \approx \hat{R}_{u,i}$
2. Using logistic matrix factorization we found an approximation of the true observations as $O_{u,i} \approx \hat{O}_{u,i}$ where $O_{u,i}$ is a binary variable representing whether the rating of (u,i) is observed or not (1 or 0, respectively). Thus, $\hat{O}_{u,i} \in (0,1)$ is the estimated probability of observing the rating of (u,i).
3. We generate the ground-truth relevance and exposure parameters as follows:
    $$ P(R_{u,i}=1) = \sigma(\hat{R}_{u,i}-\epsilon) $$
    $$ P(O_{u,i}=1) = (\hat{O}_{u,i})^p $$
    Where $\sigma$ is the sigmoid function, $\hat{R}_{u,i} \in [1,5]$ is the approximation of the true ratings, $\epsilon$ and $p$ are parameters, and $\hat{O}_{u,i}$ is the ground-truth exposure.
4. Following the probabilistic model explained in the article, we generate click variables as follows:
    $$ O_{u,i} \sim Bernoulli(P(O_{u,i}=1)) $$
    $$ R_{u,i} \sim Bernoulli(P(R_{u,i}=1)) $$
    $$ Y_{u,i} = O_{u,i} \cdot R_{u,i} $$

## Load and generate data
Using the function ```generate_sys_data()``` from data_generator.py, we can load and generate semi-synthetic data. The function takes the following arguments: ```eps, pow```. These parameters are used to generate the ground-truth relevance and exposure parameters, they correspond to $\epsilon$ and $p$ respectively.

In [ ]:
from data_generator import generate_sys_data
data = generate_sys_data()

## Arguments and parameters for the model

In [ ]:

# Args
eps = 5.0
pow_list = [1.0, 2.0, 3.0, 4.0, 5.0]
iters = 5
# Model parameters
dim = 10
lam = 1e-5
eta = 0.1
batch_size = 12
max_iters = 1000
model_name = ['oracle', 'mf', 'rmf'] # Models to compare

In [ ]:
# Define data
